> 有时我们给大模型的prompt中会带着示例 比如: 你是一个起名大师,请模仿示例起3个具有美国男孩特色的名字,示例：男孩常用名sam,女孩常用名lucy
- 上述prompt中sam和lucy就是示例
- 有时为了让结果更精确,我们会给出多组示例,但是prompt提示词是有长度限制的,langchain提供了一些工具帮助我们选择合适的示例/合适的长度的示例给到llm
 

In [2]:
import os
os.environ["OPENAI_KEY"] = "sk-bQGeRZevfQI95TPb1b1a3e80011c4f368150270f687dF163"
os.environ["OPENAI_API_BASE"] = "https://api.aihubmix.com/v1"

# 场景: 实现反义词生成器
同时会给出一些示例给大模型参考

In [3]:
#假设已经有这么多的提示词示例组
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

# 根据长度要求智能选择示例

In [6]:
#根据输入的提示词长度综合计算最终长度，智能截取或者添加提示词的示例
from langchain.prompts import PromptTemplate
# FewShotPromptTemplate: 小样本量示例选择器
from langchain.prompts import FewShotPromptTemplate
# LengthBasedExampleSelector: 基于长度的示例选择器
from langchain.prompts.example_selector import LengthBasedExampleSelector

#构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="原词与反义词 : {input} - {output}"
)

#调用长度示例选择器
example_selector = LengthBasedExampleSelector(
    #传入提示词示例组
    examples=examples,
    #传入提示词模板
    example_prompt=example_prompt,
    #设置格式化后的提示词最大长度
    max_length=25,
    #内置的get_text_length,如果默认分词计算方式不满足，可以自己扩展
    #get_text_length:Callable[[str],int] = lambda x:len(re.split("\n| ",x))
)

#使用小样本提示词模版来实现动态示例的调用
dynamic_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective} - 反义词: ",
    input_variables=["adjective"]
)

In [9]:
#小样本获得所有示例
print(dynamic_prompt.format(adjective="big"))

给出每个输入词的反义词

原词与反义词 : happy - sad

原词与反义词 : tall - short

原词与反义词 : sunny - gloomy

原词与反义词 : windy - calm

原词：big - 反义词: 


In [13]:
#如果输入长度很长，则最终输出会根据长度要求减少
long_string = "big and huge adn massive and large and gigantic and tall "
print(dynamic_prompt.format(adjective=long_string))

给出每个输入词的反义词

原词与反义词 : happy - sad

原词与反义词 : tall - short

原词：big and huge adn massive and large and gigantic and tall  - 反义词: 


# 根据输入相似度选择示例(最大边际相关性)
- MMR是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡
- MMR会首先找出与输入最相似（即余弦相似度最大）的一个样本
- 然后在迭代添加样本的过程中(一个一个的添加)，对于与已选择样本过于接近（即相似度过高）的样本进行惩罚
- MMR既能确保选出的一些样本与输入高度相关，又能保证选出的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一个平衡


In [16]:
# 后续需要使用的python内置的向量数据库
! pip install faiss-cpu 

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 10.1 MB/s eta 0:00:0000:0100:01


In [17]:
#使用MMR来检索相关示例，以使示例尽量符合输入

from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
# FAISS是langchain内置的向量数据库
from langchain.vectorstores import FAISS
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate,PromptTemplate
import os

#构造提示词模版
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="原词：{input}\n反义：{output}"
)

#调用MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性搜索
    OpenAIEmbeddings(
        openai_api_key=os.getenv("OPENAI_KEY"),
        openai_api_base=os.getenv("OPENAI_API_BASE")
    ),
    #设置使用的向量数据库是什么
    FAISS,
    #结果条数
    k=2,
)

#使用小样本模版
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective} - 反义：",
    input_variables=["adjective"]
)

In [18]:
# 当我们输入一个描述情绪的词语的时候，应该选择同样是描述情绪的一对示例组来填充提示词模版
# 同时为了符合多样性,也会返回一组无关的提示词模板
print(mmr_prompt.format(adjective="难过"))

给出每个输入词的反义词

原词：高兴
反义：悲伤

原词：tall
反义：short

原词：难过 - 反义：


# 根据输入相似度选择示例（最大余弦相似度）
- 一种常见的相似度计算方法
- 它通过计算两个向量（在这里，向量可以代表文本、句子或词语）之间的余弦值来衡量它们的相似度
- 余弦值越接近1，表示两个向量越相似
- 主要关注的是如何准确衡量两个向量的相似度

In [ ]:
# 也是一种向量数据库
! pip install chromadb

In [20]:
# 使用最大余弦相似度来检索相关示例，以使示例尽量符合输入
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import os

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词: {input} - 反义: {output}",
)
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 传入示例组.
    examples,
    # 使用openAI嵌入来做相似性搜索
    OpenAIEmbeddings(
        openai_api_key=os.getenv("OPENAI_KEY"),
        openai_api_base=os.getenv("OPENAI_API_BASE")
    ),
    # 使用Chroma向量数据库来实现对相似结果的过程存储
    Chroma,
    # 结果条数
    k=2,
)

#使用小样本提示词模板
similar_prompt = FewShotPromptTemplate(
    # 传入选择器和模板以及前缀后缀和输入变量
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词: {adjective} - 反义:",
    input_variables=["adjective"],
)

In [1]:
# 输入一个形容感觉的词语，应该查找近似的 happy/sad 示例
print(similar_prompt.format(adjective="worried"))

NameError: name 'similar_prompt' is not defined